<a href="https://colab.research.google.com/github/yamayama3689/finance/blob/main/%E6%A0%AA%E5%BC%8F%E6%83%85%E5%A0%B1%E5%8F%96%E5%BE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
import json
import pandas as pd

In [1]:
# colabのシークレットに保存しトークン情報を取得
from google.colab import userdata
J_QUANTS_EMAIL = userdata.get('J_QUANTS_EMAIL')
J_QUANTS_PASSWORD = userdata.get('J_QUANTS_PASSWORD')

In [6]:
# 上場銘柄一覧取得
# リフレッシュトークン取得

# 登録したメールアドレス、パスワードを設定
# EMAIL_ADDRESSに登録メールアドレス、PASSWORDにパスワードを入力
data={"mailaddress":J_QUANTS_EMAIL, "password":J_QUANTS_PASSWORD}

# リフレッシュトークン取得
res_ref_token = requests.post("https://api.jquants.com/v1/token/auth_user", data=json.dumps(data)) # data(Pythonの辞書形式)をjson形式に変えてやる

In [7]:
# IDトークンを取得

# リフレッシュトークンを設定
REFRESH_TOKEN = res_ref_token.json()['refreshToken']

# IDトークンを取得
res_id_token = requests.post(f"https://api.jquants.com/v1/token/auth_refresh?refreshtoken={REFRESH_TOKEN}")

{'idToken': 'eyJraWQiOiJHQXNvU2xxUzMyUktLT2lVYm1xcjU3ekdYNE1TVFhsWFBrbDNJTmhWKzNzPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiI5Y2UwZDI2Yy1jYmVmLTQzYTktYWNmNi00NDY4YWRkNDJkNGYiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiaXNzIjoiaHR0cHM6XC9cL2NvZ25pdG8taWRwLmFwLW5vcnRoZWFzdC0xLmFtYXpvbmF3cy5jb21cL2FwLW5vcnRoZWFzdC0xX0FGNjByeXJ2NCIsImNvZ25pdG86dXNlcm5hbWUiOiI5Y2UwZDI2Yy1jYmVmLTQzYTktYWNmNi00NDY4YWRkNDJkNGYiLCJvcmlnaW5fanRpIjoiZjQyZDE5YTktOWQ3Ny00MzZlLTljZDQtODQzNzc4ZTNjN2ZkIiwiYXVkIjoiNXZyN2xiOGppdThvZmhvZmJmYWxmbm1waWkiLCJldmVudF9pZCI6IjA5YTg5YzAzLTU4MjYtNDliYy05ZDcyLWNkNzU1MzQ1NmI5YiIsInRva2VuX3VzZSI6ImlkIiwiYXV0aF90aW1lIjoxNzU1MzAzNzA1LCJleHAiOjE3NTUzOTAxMjgsImlhdCI6MTc1NTMwMzcyOCwianRpIjoiZmYxOTY1YzUtYjkyNC00ZDJhLWE4MjQtZjc4NzAyMjgwNDM3IiwiZW1haWwiOiJoaXJvc2hpNDM3MjExMTJAZ21haWwuY29tIn0.EkPbAdxu2AFRWwL6SGyrEAS825pLDuTvf9DPG3VUV3JJX7wyqOxXF5CJlRqoBZlT1nxdw-vX_s66pJ7ypu2wxnJiLXRVLkmw7Hv5VWWwkAwiv3qJNj609v80B7-O8p-y2I8n6QofzwflVa_fcdKPNq_EylPtjHHyx8qtOV-oiHI4ALehzSHUPa1n_D3EftFrMjLFBJ_nj4EFlSU_ULzllhtZ-B6zc9B

In [33]:
# 全銘柄の取得
# freeプランでは、12週間より前のデータしか取得できないことに注意

# 取得したIDトークンを設定（取得から24時間まで有効）
id_token = res_id_token.json()['idToken']
headers = {'Authorization': 'Bearer {}'.format(id_token)}
DATE = '2025-05-01' # 取得可能な期間のうち、最新の月（ただし月初）の期間を指定する

# 銘柄リスト一覧を取得
res_list = requests.get(f"https://api.jquants.com/v1/listed/info?date={DATE}", headers=headers)
res_list.json()

{'info': [{'Date': '2025-05-01',
   'Code': '13010',
   'CompanyName': '極洋',
   'CompanyNameEnglish': 'KYOKUYO CO.,LTD.',
   'Sector17Code': '1',
   'Sector17CodeName': '食品',
   'Sector33Code': '0050',
   'Sector33CodeName': '水産・農林業',
   'ScaleCategory': 'TOPIX Small 2',
   'MarketCode': '0111',
   'MarketCodeName': 'プライム'},
  {'Date': '2025-05-01',
   'Code': '13050',
   'CompanyName': '大和アセットマネジメント株式会社\u3000ｉＦｒｅｅＥＴＦ\u3000ＴＯＰＩＸ（年１回決算型）',
   'CompanyNameEnglish': 'iFreeETF TOPIX (Yearly Dividend Type)',
   'Sector17Code': '99',
   'Sector17CodeName': 'その他',
   'Sector33Code': '9999',
   'Sector33CodeName': 'その他',
   'ScaleCategory': '-',
   'MarketCode': '0109',
   'MarketCodeName': 'その他'},
  {'Date': '2025-05-01',
   'Code': '13060',
   'CompanyName': '野村アセットマネジメント株式会社\u3000ＮＥＸＴ\u3000ＦＵＮＤＳ\u3000ＴＯＰＩＸ連動型上場投信',
   'CompanyNameEnglish': 'NEXT FUNDS TOPIX Exchange Traded Fund',
   'Sector17Code': '99',
   'Sector17CodeName': 'その他',
   'Sector33Code': '9999',
   'Sector33CodeName': 'その他',


In [29]:
# pandasのデータフレームに変換
df_code_list = pd.DataFrame(res_list.json()['info'])

In [32]:
# データの確認
df_code_list.head()
# Date:取得日付
# Code:銘柄コード
# CompanyName:会社名
# Sector17Code:17業種コード
# Sector33Code:33業種コード
# ScaleCategory:規模コード
# MarketCode:市場区分コード その他にはETFが含まれている

,Date,Code,CompanyName,CompanyNameEnglish,Sector17Code,Sector17CodeName,Sector33Code,Sector33CodeName,ScaleCategory,MarketCode,MarketCodeName
0,2025-05-01,13010,極洋,"KYOKUYO CO.,LTD.",1,食品,0050,水産・農林業,TOPIX Small 2,0111,プライム
1,2025-05-01,13050,大和アセットマネジメント株式会社 ｉＦｒｅｅＥＴＦ ＴＯＰＩＸ（年１回決算型）,iFreeETF TOPIX (Yearly Dividend Type),99,その他,9999,その他,-,0109,その他
2,2025-05-01,13060,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,NEXT FUNDS TOPIX Exchange Traded Fund,99,その他,9999,その他,-,0109,その他
3,2025-05-01,13080,日興アセットマネジメント株式会社 上場インデックスファンドＴＯＰＩＸ,Listed Index Fund TOPIX,99,その他,9999,その他,-,0109,その他
4,2025-05-01,13090,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０...,NEXT FUNDS ChinaAMC SSE50 Index Exchange Trade...,99,その他,9999,その他,-,0109,その他


In [31]:
df_code_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4415 entries, 0 to 4414
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Date                4415 non-null   object
 1   Code                4415 non-null   object
 2   CompanyName         4415 non-null   object
 3   CompanyNameEnglish  4415 non-null   object
 4   Sector17Code        4415 non-null   object
 5   Sector17CodeName    4415 non-null   object
 6   Sector33Code        4415 non-null   object
 7   Sector33CodeName    4415 non-null   object
 8   ScaleCategory       4415 non-null   object
 9   MarketCode          4415 non-null   object
 10  MarketCodeName      4415 non-null   object
dtypes: object(11)
memory usage: 379.5+ KB


In [ ]:
# 会社名の全角スペース（\u3000）を半角スペースに置き換え
df_code_list['CompanyName'] = df_code_list['CompanyName'].str.replace('\u3000',' ')

In [26]:
# プライム市場のみを抽出する
df_code_list_prime = df_code_list[(df_code_list['MarketCode'] == '0111')]

In [27]:
df_code_list_prime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1636 entries, 0 to 4414
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Date                1636 non-null   object
 1   Code                1636 non-null   object
 2   CompanyName         1636 non-null   object
 3   CompanyNameEnglish  1636 non-null   object
 4   Sector17Code        1636 non-null   object
 5   Sector17CodeName    1636 non-null   object
 6   Sector33Code        1636 non-null   object
 7   Sector33CodeName    1636 non-null   object
 8   ScaleCategory       1636 non-null   object
 9   MarketCode          1636 non-null   object
 10  MarketCodeName      1636 non-null   object
dtypes: object(11)
memory usage: 153.4+ KB


In [28]:
df_code_list_prime

,Date,Code,CompanyName,CompanyNameEnglish,Sector17Code,Sector17CodeName,Sector33Code,Sector33CodeName,ScaleCategory,MarketCode,MarketCodeName
0,2025-05-01,13010,極洋,"KYOKUYO CO.,LTD.",1,食品,0050,水産・農林業,TOPIX Small 2,0111,プライム
18,2025-05-01,13320,ニッスイ,Nissui Corporation,1,食品,0050,水産・農林業,TOPIX Mid400,0111,プライム
19,2025-05-01,13330,マルハニチロ,Maruha Nichiro Corporation,1,食品,0050,水産・農林業,TOPIX Mid400,0111,プライム
39,2025-05-01,13750,ユキグニファクトリー,"YUKIGUNI FACTORY CO.,LTD.",1,食品,0050,水産・農林業,TOPIX Small 2,0111,プライム
41,2025-05-01,13770,サカタのタネ,SAKATA SEED CORPORATION,1,食品,0050,水産・農林業,TOPIX Small 1,0111,プライム
...,...,...,...,...,...,...,...,...,...,...,...
4407,2025-05-01,99870,スズケン,"SUZUKEN CO.,LTD.",13,商社・卸売,6050,卸売業,TOPIX Mid400,0111,プライム
4408,2025-05-01,99890,サンドラッグ,"SUNDRUG CO.,LTD.",14,小売,6100,小売業,TOPIX Mid400,0111,プライム
4409,2025-05-01,99900,サックスバー ホールディングス,SAC'S BAR HOLDINGS INC.,14,小売,6100,小売業,TOPIX Small 2,0111,プライム
4410,2025-05-01,99910,ジェコス,GECOSS CORPORATION,13,商社・卸売,6050,卸売業,TOPIX Small 2,0111,プライム
